In [29]:
from pgtools.gff_parser import parse_GFFs_dir, Gff
from typing import Dict
import os

In [125]:
"(hdgrdr)".strip("()")

'hdgrdr'

# CSV with contig lens

csv such that:
genome, contig_name, contig_len, no. of annotations on the contig

In [2]:
gffs_dir = "/home/pampuch/studia/magisterka/test_data/gffs/"

In [15]:
kleb_gff = parse_GFFs_dir(os.path.join(gffs_dir, "klebsiella_subset/gff"), gff_simple=False)

In [35]:
def summary_dict(genome_gffs: Dict[str, Gff]):
    gffs_summary = {}
    for genome, gff in genome_gffs.items():
        genome_scaffolds = {scaff.name : {"annot_n": 0, "length": None} for scaff in gff.scaffolds}
        for scaff in gff.scaffolds:
            genome_scaffolds[scaff.name]["length"] = scaff.length
        for cds in gff.cds:
            genome_scaffolds[cds.scaffold.name]["annot_n"] += 1
        # print(genome_scaffolds)
        gffs_summary[genome] = genome_scaffolds
    return(gffs_summary)
summary_dict(kleb_gff)
    

{'5235_5_12': {'contig00001': {'annot_n': 62, 'length': 64326},
  'contig00002': {'annot_n': 63, 'length': 60612},
  'contig00003': {'annot_n': 47, 'length': 53131},
  'contig00004': {'annot_n': 47, 'length': 42928},
  'contig00005': {'annot_n': 32, 'length': 40521},
  'contig00006': {'annot_n': 44, 'length': 40109},
  'contig00007': {'annot_n': 40, 'length': 36923},
  'contig00008': {'annot_n': 37, 'length': 36600},
  'contig00009': {'annot_n': 33, 'length': 36365},
  'contig00010': {'annot_n': 33, 'length': 35201},
  'contig00011': {'annot_n': 31, 'length': 33934},
  'contig00012': {'annot_n': 34, 'length': 33337},
  'contig00013': {'annot_n': 32, 'length': 32855},
  'contig00014': {'annot_n': 33, 'length': 32199},
  'contig00015': {'annot_n': 29, 'length': 31282},
  'contig00016': {'annot_n': 22, 'length': 30241},
  'contig00017': {'annot_n': 28, 'length': 29843},
  'contig00018': {'annot_n': 32, 'length': 29837},
  'contig00019': {'annot_n': 26, 'length': 29361},
  'contig00020': {

In [36]:
kleb_dict = summary_dict(kleb_gff)

In [48]:
df_rows = []
for genome, genome_contigs in kleb_dict.items():
    # print(genome_contigs)\
    contigs_sorted = sorted([contig for contig in genome_contigs], key=lambda x: (genome_contigs[x]["length"], genome_contigs[x]["annot_n"]), reverse=True)
    for contig in contigs_sorted:
        # print(genome_contigs[contig])
        df_rows.append([genome, contig, genome_contigs[contig]["length"], genome_contigs[contig]["annot_n"]])


In [50]:
import pandas as pd

In [51]:
df = pd.DataFrame(df_rows, columns = ["genome", "contig", "contig length", "no. of annotations on contig"])

In [52]:
df

,genome,contig,contig length,no. of annotations on contig
0,5235_5_12,contig00001,64326,62
1,5235_5_12,contig00002,60612,63
2,5235_5_12,contig00003,53131,47
3,5235_5_12,contig00004,42928,47
4,5235_5_12,contig00005,40521,32
...,...,...,...,...
15603,5235_6_6,contig01171,56,0
15604,5235_6_6,contig01172,56,0
15605,5235_6_6,contig01173,56,0
15606,5235_6_6,contig01174,56,0


In [53]:
df["genome"].nunique()

20

In [54]:
def dict_to_sorted_rows(summary_dictionary):
    df_rows = []
    for genome, genome_contigs in summary_dictionary.items():
        # print(genome_contigs)\
        contigs_sorted = sorted([contig for contig in genome_contigs], key=lambda x: (genome_contigs[x]["length"], genome_contigs[x]["annot_n"]), reverse=True)
        for contig in contigs_sorted:
            # print(genome_contigs[contig])
            df_rows.append([genome, contig, genome_contigs[contig]["length"], genome_contigs[contig]["annot_n"]])
    return df_rows

In [60]:
def gffs_summary_csv(dataset, gff_dir):
    gffs = parse_GFFs_dir(os.path.join(gffs_dir, dataset, "gff"), gff_simple=False)
    gffs_summary = summary_dict(gffs)
    df_rows = dict_to_sorted_rows(gffs_summary)
    df = pd.DataFrame(df_rows, columns = ["genome", "contig", "contig length", "no. of annotations on contig"])
    df.to_csv(os.path.join("/home/pampuch/studia/magisterka/gffs_summary", dataset + ".csv"), index=False)

In [61]:
for dataset in os.listdir(gffs_dir):
    gffs_summary_csv(dataset, gffs_dir)

In [97]:
df_kleb = pd.read_csv("/home/pampuch/studia/magisterka/gffs_summary/klebsiella_subset.csv")

df_kleb.groupby(["genome"], as_index=False)["contig length"].idxmax()

,genome,contig length
0,5150_1_3,14057
1,5150_2_2,7662
2,5150_3_5,13363
3,5151_2_6,1347
4,5151_6_6,2043
5,5193_1_5,10618
6,5193_2_6,9381
7,5193_8_2,10078
8,5197_2_1,3407
9,5197_7_4,12995


In [95]:
df_kleb

,genome,contig,contig length,no. of annotations on contig
0,5235_5_12,contig00001,64326,62
1,5235_5_12,contig00002,60612,63
2,5235_5_12,contig00003,53131,47
3,5235_5_12,contig00004,42928,47
4,5235_5_12,contig00005,40521,32
...,...,...,...,...
15603,5235_6_6,contig01171,56,0
15604,5235_6_6,contig01172,56,0
15605,5235_6_6,contig01173,56,0
15606,5235_6_6,contig01174,56,0


In [94]:
df_kleb.groupby(["genome"], as_index=False)["contig length"].max()

,genome,contig length
0,5150_1_3,520083
1,5150_2_2,453849
2,5150_3_5,309492
3,5151_2_6,309177
4,5151_6_6,392568
5,5193_1_5,304092
6,5193_2_6,260207
7,5193_8_2,419990
8,5197_2_1,452471
9,5197_7_4,382127


In [101]:
max_idx = list(df_kleb.groupby(["genome"], as_index=False)["contig length"].idxmax()["contig length"])

In [102]:
max_idx

[14057,
 7662,
 13363,
 1347,
 2043,
 10618,
 9381,
 10078,
 3407,
 12995,
 13779,
 7811,
 2330,
 0,
 11024,
 12107,
 6427,
 14433,
 4014,
 4535]

In [110]:
df_best_contigs = df_kleb.loc[max_idx]

In [116]:
df_best_contigs.sort_values(["contig length"], ascending=False)

,genome,contig,contig length,no. of annotations on contig
14057,5150_1_3,contig00001,520083,506
7662,5150_2_2,contig00001,453849,440
3407,5197_2_1,contig00001,452471,443
10078,5193_8_2,contig00001,419990,395
2043,5151_6_6,contig00001,392568,383
13779,5197_7_5,contig00001,383525,367
12995,5197_7_4,contig00001,382127,338
13363,5150_3_5,contig00001,309492,304
1347,5151_2_6,contig00001,309177,294
10618,5193_1_5,contig00001,304092,261


In [121]:
def df_top_contigs(dataset_csv):
    df = pd.read_csv(f"/home/pampuch/studia/magisterka/gffs_summary/{dataset}.csv")
    max_idx = list(df.groupby(["genome"], as_index=False)["contig length"].idxmax()["contig length"])
    df.loc[max_idx].to_csv(os.path.join("/home/pampuch/studia/magisterka/gffs_summary/top_per_genome/",f"{dataset}_top_for_genome.csv"))
    return df.loc[max_idx]

In [122]:
df_top_contigs("GPSC_subset")

,genome,contig,contig length,no. of annotations on contig
1127,14520_5_48.velvet,ERS525028_SC_contig000001,299012,315
1976,15841_4_17.velvet,ERS639860_SC_contig000001,199775,208
465,15841_4_38.velvet,ERS639881_SC_contig000001,275841,292
2262,17138_5_34.velvet,ERS653856_SC_contig000001,123109,113
1390,17794_8_81.velvet,ERS719881_SC_contig000001,204105,200
2210,17870_7_102.velvet,ERS725745_SC_contig000001,222230,219
0,17870_7_11.velvet,ERS725493_SC_contig000001,393383,396
1235,17870_7_77.velvet,ERS725456_SC_contig000001,335291,358
1810,18090_1_123.velvet,ERS740691_SC_contig000001,233604,248
1005,19084_7_49.velvet,ERS1021351_SC_contig000001,215367,229


In [123]:
for dataset in os.listdir(gffs_dir):
    df_top_contigs(dataset)